In [35]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd
import xarray as xr
import math
from datetime import datetime,date
import numpy as np
import os

In [5]:
allreg = 1  # generate all from regular gridded files
ireg   = 1  # generate all from irregular gridded files 

#### Load observations

In [6]:
mosasfs30met = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/maltem/MOSAiC/mosasfs30met.level2.10min.all.nc")
mosasfs40met = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/maltem/MOSAiC/mosasfs40met.level2.10min.all.nc")
mosasfs50met = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/maltem/MOSAiC/mosasfs50met.level2.10min.all.nc")

In [31]:
idir = '/home/maltem/store/nowwind/MOSAiC/'
idir = '/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive/'


#### Interpolate to hourly values

In [7]:
mosasfs30met_1h = mosasfs30met.resample(time="1H").interpolate("linear")
mosasfs40met_1h = mosasfs40met.resample(time="1H").interpolate("linear")
mosasfs50met_1h = mosasfs50met.resample(time="1H").interpolate("linear")

### All MOSAiC dates & Check if AROME file exists

In [50]:



for statnr in ['30', '40', '50']:  #statnr='30'; # MET station number 30, 40, 50 

    strd=[]  # 'surface_downwelling_longwave_flux_in_air'     
    strn=[]  # 'surface_net_downward_longwave_flux_wrt_time'                      
    t2m=[]; u10m=[]; v10m=[]; sp=[]; date=[]
    
    if statnr=='30':
        indata = mosasfs30met_1h    
    elif statnr=='40':
        indata = mosasfs40met_1h 
    elif statnr== '50':
        indata = mosasfs50met_1h     
    
    for t in range(indata.sizes['time']):
        tdate   = indata.time[t].values
        tdatep1h = indata.time[t].values+np.timedelta64(1, 'h')
        tdatem6h = indata.time[t].values-np.timedelta64(6, 'h') # Ommit the first 6 hours of the fc
        
        datet.append(tdate)
        
        yy = str("%04d" % pd.to_datetime(tdatem6h).year)
        mm = str("%02d" % pd.to_datetime(tdatem6h).month)
        dd = str("%02d" % pd.to_datetime(tdatem6h).day)
        
        ifile = idir + '/' + yy + '/' + mm + '/' + dd + '/00/' + 'cy43_mosaic_drift_'+yy+mm+dd+'00_grib2_fp.nc'
        #print (ifile)
        useifile=[]
        if (os.path.isfile(ifile)):   # IF FILE EXISTS 
            useifile=ifile  
        else:                         # CHECK IF FILE OF PREVIOUS DAY EXISTS
            ddm1 = str("%02d" % pd.to_datetime(tdatem6h-np.timedelta64(1, 'D')).day)
            ifile = idir + '/' + yy + '/' + mm + '/' + ddm1 + '/00/' + 'cy43_mosaic_drift_'+yy+mm+ddm1+'00_grib2_fp.nc'
            if (os.path.isfile(ifile)):
                print('previous file exists')
                useifile=file
        
        if not useifile: # If no model data exists
            
            strd.append(np.nan);   strn.append(np.nan);
            t2m.append(np.nan);    u10m.append(np.nan);    v10m.append(np.nan);
            sp.append(np.nan);
        else:            # If model data exists
            
            AROMEf = xr.open_mfdataset(useifile)
            latpos = indata.lat[t].values; lonpos = indata.lon[t].values # Find position
            [ix,jx] = findindex(np.array(AROMEf.latitude),np.array(AROMEf.longitude),latpos,lonpos)
            
            #De-accumulate
            AROF=np.array(AROMEf.sel(time=tdatep1h, method='nearest').surface_downwelling_longwave_flux_in_air) - np.array(AROMEf.sel(time=tdate, method='nearest').surface_downwelling_longwave_flux_in_air)
            strd.append(AROF[0,ix,jx])
            #De-accumulate
            AROF=np.array(AROMEf.sel(time=tdatep1h, method='nearest').surface_net_downward_longwave_flux_wrt_time) - np.array(AROMEf.sel(time=tdate, method='nearest').surface_net_downward_longwave_flux_wrt_time)
            strn.append(AROF[0,ix,jx])

            AROF=np.array(AROMEf.sel(time=tdate, method='nearest').air_temperature_2m)
            t2m.append(AROF[0,ix,jx])          
            AROF=np.array(AROMEf.sel(time=tdate, method='nearest').x_wind_10m)
            u10m.append(AROF[0,ix,jx])
            AROF=np.array(AROMEf.sel(time=tdate, method='nearest').y_wind_10m)
            v10m.append(AROF[0,ix,jx])          
            AROF=np.array(AROMEf.sel(time=tdate, method='nearest').air_pressure_at_sea_level)
            sp.append(AROF[0,ix,jx])
            
    dfout = pd.DataFrame(list(zip(datet,strd,strn,t2m,u10m,v10m,sp)),
                             columns =['datetime','strd','strn','t2m','u10m','v10m','sp'])

    pd.DataFrame(dfout).to_csv("mosasfs"+statnr+"_AROME.csv",index=None)            

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/14/00/cy43_mosaic_drift_2020011400_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/03/16/00/cy43_mosaic_drift_2020031600_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/14/00/cy43_mosaic_drift_2019121400_grib2_fp.nc
prev

...

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/17/00/cy43_mosaic_drift_2020011700_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/09/00/cy43_mosaic_drift_2019120900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/11/00/cy43_mosaic_drift_2019121100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/11/00/cy43_mosaic_drift_2019121100_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2019/12/19/00/cy43_mosaic_drift_2019121900_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/11/00/cy43_mosaic_drift_2020011100_grib2_fp.nc
prev

previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
previous file exists
/lustre/storeB/project/nwp/alertness/arome_experiments/teresav/cy43_mosaic_drift/archive//2020/01/27/00/cy43_mosaic_drift_2020012700_grib2_fp.nc
prev